In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import torch
from torch import nn
import gin
from sparse_causal_model_learner_rl.trainable.gumbel_switch import WithInputSwitch, sample_from_logits_simple, LearnableSwitchSimple
gin.enter_interactive_mode()

In [2]:
def get_model():
    m = torch.nn.Sequential(
        torch.nn.Linear(in_features=25, out_features=60,),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=60, out_features=60),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=60, out_features=60),
        torch.nn.ReLU(),
        torch.nn.Linear(in_features=60, out_features=1),
    )
    return m

In [39]:
torch.jit.fork

AttributeError: module 'torch.jit' has no attribute 'fork'

In [11]:
n_models = 25
data = torch.randn(1000, 25)

In [34]:
class AllModels(nn.Module):
    def __init__(self):
        super(AllModels, self).__init__()
        
        self.models = []
        self.model_list = []
        for m in range(n_models):
            m_name = 'm%02d' % m
            model = get_model()
            setattr(self, m_name, model)
            self.models.append(m_name)
            self.model_list.append(model)
       
#     @torch.jit.script
    def forward_parallel(self, data):
        models = self.model_list
        futs = [torch.jit.fork(model.forward, data) for model in models]
        results = [torch.jit.wait(fut) for fut in futs]
        return torch.cat(results, dim=1)
    
    def forward(self, data, do_parallel=False):
        if do_parallel:
            return self.forward_parallel(data)
        results = [getattr(self, m)(data) for m in self.models]
        return torch.cat(results, dim=1)
    
# @torch.jit.script
# def forward_one(i, data):
#     fut = torch.jit._fork(models[i].forward, data)
#     z = torch.jit._wait(fut)
#     return z

In [35]:
M = AllModels()

In [38]:
%timeit M(data, do_parallel=True)

AttributeError: module 'torch.jit' has no attribute 'fork'

In [36]:
%timeit M(data)

235 ms ± 7.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


AttributeError: module 'torch.jit' has no attribute 'fork'

In [5]:
%timeit models[0](data)

8.48 ms ± 507 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit [m(data) for m in models]

224 ms ± 9.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
8.7*25

217.49999999999997

In [8]:
# a simple python loop does it SEQUENTIALLY

In [9]:
from torch.nn.parallel import parallel_apply

In [ ]:
%timeit parallel_apply(models, [data] * n_models)